In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import os
import csv
from pprint import pprint

In [2]:
#listing API urls to use in data pulls
base_url = 'https://public.opendatasoft.com/api/records/1.0/'
records_url_listings = 'search/?dataset=airbnb-listings'
download_url_listings = 'download/?dataset=airbnb-listings'
records_url_reviews = 'search/?dataset=airbnb-reviews'
download_url_reviews = 'download/?dataset=airbnb-reviews'

In [3]:
city_pop_file = 'Resources/city_populations.csv'
city_pop_df = pd.read_csv(city_pop_file)
city_pop_df.head()

,rank,City,Country,population_2019
0,1,Tokyo,Japan,37435192
1,2,Delhi,India,29399140
2,3,Shanghai,China,26317104
3,4,Sao Paulo,Brazil,21846508
4,5,Mexico City,Mexico,21671908


In [4]:
city_list = city_pop_df['City'].head(100)
city_list

0            Tokyo
1            Delhi
2         Shanghai
3        Sao Paulo
4      Mexico City
          ...     
95           Kabul
96           Hefei
97        Tel Aviv
98          Recife
99    Shijiazhuang
Name: City, Length: 100, dtype: object

In [25]:
listing_df = pd.DataFrame()
#city='Austin'
i=0
for city in city_list:
    get_url = base_url+records_url_listings+f'&facet=city&refine.city={city}&rows=10000&fields=country,city,name,host_name,street,property_type,accommodates,bedrooms,bathrooms,beds,price,weekly_price,monthly_price,security_deposit,cleaning_fee,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,listing_url'
    response = requests.get(get_url).json()
    for x in range(len(response['records'])):
        listing_df = listing_df.append(json_normalize(response['records'][x]['fields']))
    print(f'Finished listings in {city}, {i} of {len(city_list)}')
    i+=1
    
    
listing_df.head()

Finished listings in Tokyo, 0 of 100
Finished listings in Delhi, 1 of 100
Finished listings in Shanghai, 2 of 100
Finished listings in Sao Paulo, 3 of 100
Finished listings in Mexico City, 4 of 100
Finished listings in Cairo, 5 of 100
Finished listings in Dhaka, 6 of 100
Finished listings in Mumbai, 7 of 100
Finished listings in Beijing, 8 of 100
Finished listings in Osaka, 9 of 100
Finished listings in Karachi, 10 of 100
Finished listings in Chongqing, 11 of 100
Finished listings in Buenos Aires, 12 of 100
Finished listings in Istanbul, 13 of 100
Finished listings in Kolkata, 14 of 100
Finished listings in Lagos, 15 of 100
Finished listings in Kinshasa, 16 of 100
Finished listings in Manila, 17 of 100
Finished listings in Tianjin, 18 of 100
Finished listings in Rio de Janeiro, 19 of 100
Finished listings in Guangzhou, 20 of 100
Finished listings in Moscow, 21 of 100
Finished listings in Lahore, 22 of 100
Finished listings in Shenzhen, 23 of 100
Finished listings in Bangalore, 24 of 10

,city,bathrooms,name,minimum_nights,country,price,bedrooms,accommodates,beds,number_of_reviews,street,host_name,maximum_nights,listing_url,property_type,review_scores_rating,monthly_price,weekly_price,cleaning_fee,security_deposit
0,Shenzhen,1.0,Bears & Deers Youth Hostel - 3 BunkBed Female ...,1,China,88.0,1.0,6,6.0,0,"福强路, Shenzhen, Guangdong, China",Katniss,1125,https://www.airbnb.com/rooms/13999779,Apartment,NaN,NaN,NaN,NaN,NaN
0,Shenzhen,1.0,welcome gay friends and nudists,1,China,184.0,1.0,1,1.0,1,"沿河南路, Shenzhen, Guangdong, China",Wallace,1125,https://www.airbnb.com/rooms/1631158,Apartment,80.0,NaN,NaN,NaN,NaN
0,Shenzhen,1.0,a Secret sky garden Next to Luohu Perthden Hotel,1,China,264.0,1.0,2,1.0,0,"Shenzhen, Guangdong 518001, China",Oscar,8,https://www.airbnb.com/rooms/14063808,Apartment,NaN,NaN,NaN,NaN,NaN
0,Shenzhen,1.0,宽敞舒适豪华两房,1,China,487.0,2.0,5,2.0,0,"Shenzhen, Guangdong 518031, China",诗凌,1125,https://www.airbnb.com/rooms/14343031,Apartment,NaN,NaN,NaN,NaN,NaN
0,Shenzhen,2.0,福田口岸地铁口，得宝青年公寓女生观景四人间(有独立卫浴),1,China,88.0,1.0,4,4.0,2,"Shenzhen, Guangdong, China",Fanny,1125,https://www.airbnb.com/rooms/13008645,Apartment,90.0,2399.0,624.0,NaN,NaN


In [53]:
listing_df.reset_index(drop=True, inplace = True)

In [59]:
listing_df.columns

Index(['city', 'bathrooms', 'name', 'minimum_nights', 'country', 'price',
       'bedrooms', 'accommodates', 'beds', 'number_of_reviews', 'street',
       'host_name', 'maximum_nights', 'listing_url', 'property_type',
       'review_scores_rating', 'monthly_price', 'weekly_price', 'cleaning_fee',
       'security_deposit', 'city_id'],
      dtype='object')

In [58]:
city_id_col=[]
for z in range(len(city_list)):
    for y in range(len(listing_df)):
        if listing_df['city'][y]==city_list[z] :
            city_id_col.append(z)
            
listing_df['city_id']=city_id_col

In [66]:
listing_df.country.unique()

array(['China', 'France', 'United Kingdom', 'Hong Kong', 'Spain',
       'Canada', 'Australia', 'Italy'], dtype=object)

In [67]:
listing_df.city.unique()

array(['Shenzhen', 'Paris', 'London', 'Hong Kong', 'Madrid', 'Toronto',
       'Barcelona', 'Qingdao', 'Alexandria', 'Melbourne', 'Sydney',
       'Rome', 'Montreal'], dtype=object)

In [56]:
country_id_col=[]
for w in range(len(country_list)):
    for t in range(len(listing_df)):
        if listing_df['city'][t]==city_list[w] :
            city_id_col.append(w)
            
listing_df['city_id']=country_id_col

'Shenzhen'

In [27]:
listing_output_file = 'Resources/listings.csv'

In [26]:
#listing_df.to_csv(listing_output_file, index=false, header=true)

65486

In [63]:
listing_df

,city,bathrooms,name,minimum_nights,country,price,bedrooms,accommodates,beds,number_of_reviews,...,host_name,maximum_nights,listing_url,property_type,review_scores_rating,monthly_price,weekly_price,cleaning_fee,security_deposit,city_id
0,Shenzhen,1.0,Bears & Deers Youth Hostel - 3 BunkBed Female ...,1,China,88.0,1.0,6,6.0,0,...,Katniss,1125,https://www.airbnb.com/rooms/13999779,Apartment,NaN,NaN,NaN,NaN,NaN,23
1,Shenzhen,1.0,welcome gay friends and nudists,1,China,184.0,1.0,1,1.0,1,...,Wallace,1125,https://www.airbnb.com/rooms/1631158,Apartment,80.0,NaN,NaN,NaN,NaN,23
2,Shenzhen,1.0,a Secret sky garden Next to Luohu Perthden Hotel,1,China,264.0,1.0,2,1.0,0,...,Oscar,8,https://www.airbnb.com/rooms/14063808,Apartment,NaN,NaN,NaN,NaN,NaN,23
3,Shenzhen,1.0,宽敞舒适豪华两房,1,China,487.0,2.0,5,2.0,0,...,诗凌,1125,https://www.airbnb.com/rooms/14343031,Apartment,NaN,NaN,NaN,NaN,NaN,23
4,Shenzhen,2.0,福田口岸地铁口，得宝青年公寓女生观景四人间(有独立卫浴),1,China,88.0,1.0,4,4.0,2,...,Fanny,1125,https://www.airbnb.com/rooms/13008645,Apartment,90.0,2399.0,624.0,NaN,NaN,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65481,Montreal,NaN,AMAZING! THE BEST PLACE IN MONTREAL,2,Canada,34.0,1.0,2,1.0,55,...,Silva,1125,https://www.airbnb.com/rooms/712156,Apartment,95.0,960.0,266.0,NaN,NaN,92
65482,Montreal,1.0,"Montréal, Plateau Mont-Royal Loft",30,Canada,79.0,1.0,2,1.0,16,...,Jacques,1125,https://www.airbnb.com/rooms/482691,Apartment,90.0,1350.0,495.0,30.0,500.0,92
65483,Montreal,1.0,"Great DEAL, fantastic location !",4,Canada,79.0,1.0,3,1.0,10,...,Matthieu & Roisin,365,https://www.airbnb.com/rooms/707743,Apartment,91.0,NaN,NaN,NaN,NaN,92
65484,Montreal,1.0,FRESH NEW APARTMENT IN THE PLATEAU,2,Canada,176.0,3.0,6,3.0,36,...,Daniel,365,https://www.airbnb.com/rooms/1346097,Apartment,89.0,NaN,NaN,80.0,300.0,92


,monthly_price,city,bathrooms,name,minimum_nights,country,price,bedrooms,accommodates,beds,number_of_reviews,street,cleaning_fee,host_name,maximum_nights,listing_url,review_scores_rating,property_type,security_deposit
0,7000.0,Austin,1.0,"Charming, Authentically Austin",1,United States,259,2,6,2,16,"Dawson Road, Austin, TX 78704, United States",25.0,Park Terrace,30,https://www.airbnb.com/rooms/4994857,89.0,Apartment,100.0
0,NaN,Austin,1.0,Historic 2BD in Travis Heights,1,United States,800,2,4,2,0,"Alameda Drive, Austin, TX 78704, United States",NaN,Charles,1125,https://www.airbnb.com/rooms/10546343,NaN,House,NaN
